In [1]:
import pandas as pd

In [7]:
pd.set_option("display.max_rows", None)

In [3]:
calls_df = pd.read_csv("https://storage.googleapis.com/cs163-seniorproject.appspot.com/dataset/911_calls/911_calls_clean.csv")

In [5]:
print(calls_df["ADDRESS"].nunique())

85772


In [15]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import folium
import time

# calls_df = pd.read_csv("your_911_calls.csv")

# Get top 1000 frequent addresses
top_addresses = calls_df["ADDRESS"].value_counts().head(1000).index.tolist()

# Normalize addresses
def normalize_address(addr):
    if pd.notnull(addr):
        addr = str(addr).replace('[', '').replace(']', '')
        if '-' in addr and any(char.isdigit() for char in addr.split('-')[0]):
            parts = addr.split(' ')
            range_part = parts[0]
            if '-' in range_part:
                nums = range_part.split('-')
                try:
                    midpoint = str((int(nums[0]) + int(nums[1])) // 2)
                    addr = midpoint + ' ' + ' '.join(parts[1:])
                except:
                    pass
        addr = addr.replace('&', 'and')
        return addr + ", San Jose, CA"
    return None

normalized_addresses = [normalize_address(addr) for addr in top_addresses]

# Safe geocode function
geolocator = Nominatim(user_agent="sj911-mapper", timeout=10)

def safe_geocode(address, retries=3):
    for i in range(retries):
        try:
            return geolocator.geocode(address)
        except (GeocoderTimedOut, GeocoderUnavailable):
            print(f"Retry {i+1}/{retries} for address: {address}")
            time.sleep(2)
    print(f"Failed to geocode: {address}")
    return None

# Geocode and collect results
locations = []

for addr in normalized_addresses:
    loc = safe_geocode(addr)
    if loc:
        locations.append({
            "address": addr,
            "lat": loc.latitude,
            "lon": loc.longitude
        })
    time.sleep(1)  # Rate limiting

# Save geocoded addresses to cache
loc_df = pd.DataFrame(locations)

# Plot on folium map
map_sj = folium.Map(location=[37.3382, -121.8863], zoom_start=12)

for _, row in loc_df.iterrows():
    folium.Marker(
        location=[row["lat"], row["lon"]],
        popup=row["address"],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(map_sj)

map_sj
